<a href="https://colab.research.google.com/github/paranjaa/ece1508-student-depression-project/blob/main/ECE1508_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 1508 Final Project #
## Training MLPs to recognize Depression ##

Seperated out the coding individually, each person experimented w/ different architectures and models before putting everything together

### Imports

In [ ]:
# for importing the data
from google.colab import files


#import each one of the libraries
import pandas as pd
import torch
import numpy as np

# used put tensorflow and pytorch
import tensorflow as tf
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


#for DL models
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split #split train and tes sets
from sklearn.preprocessing import StandardScaler #to scale our data
from sklearn.metrics import accuracy_score
#we try these three models first
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

# for plotting graphs
import matplotlib.pyplot as plt

#for prompting w/ chat gpt
import openai
import time

Import dataset either from file upload (at https://www.kaggle.com/datasets/hopesb/student-depression-dataset)

(Or from google drive, seems like upload doesn't properly w/ mobile)

In [ ]:
uploaded = files.upload()
df = pd.read_csv("Student Depression Dataset.csv")

# from google.colab import drive
# drive.mount("/content/gdrive")
# df = pd.read_csv("gdrive/My Drive/ECE1508/Project/Student_Depression_Dataset.csv")



Saving Student Depression Dataset.csv to Student Depression Dataset.csv


#### Dataset examination/correlation ####

Check the contents of the dataset
(it has 27900 students, mostly in undergrad, about _% are depressed

In [ ]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                         27901 non-null  object 
 11  Di

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [ ]:
print ((df['Depression'].value_counts()) / len(df) * 100)
# can see that 58% of the students are depressed, higher than the study in the presentation

Depression
1    58.549873
0    41.450127
Name: count, dtype: float64


Using crosstab on several columns to see what rows have results that correlate strongly with depression

In [ ]:
# Using crosstab on each row to look for correlation w/ depression
pd.crosstab(df['Gender'], df['Depression'], normalize='index') * 100
# Depression rates amongst genders isn't that different (figured there'd be some difference?) not useful for checking

Depression,0,1
Gender,,
Female,41.549296,58.450704
Male,41.371326,58.628674


In [ ]:
pd.crosstab(df['Academic Pressure'], df['Depression'], normalize='index') * 100
# propbably expected, but increasing rates of depression for higher levels of academic pressure
# kind of unsure why 0.0 is higher than 1.0? Possibly from null answers, or sampling confusion?

Depression,0,1
Academic Pressure,,
0.0,55.555556,44.444444
1.0,80.587378,19.412622
2.0,62.517951,37.482049
3.0,39.841865,60.158135
4.0,23.860330,76.139670
5.0,13.913596,86.086404


In [ ]:
# skipping age/city/cgpa -> not in discrete enough groups to correlate (maybe put in categories? later)
# skipping profession/job pressure -> not enough people put non-zero answers (almost everyone polled is a student?)

In [ ]:
pd.crosstab(df['Sleep Duration'], df['Depression'], normalize='index') * 100
# Oddly enough, also not correlated strongly w/ depression. less than 5 hours is highest, but not by much, and 7-8 hours is second place?

Depression,0,1
Sleep Duration,,
5-6 hours,43.118227,56.881773
7-8 hours,40.498230,59.501770
Less than 5 hours,35.487365,64.512635
More than 8 hours,49.073461,50.926539
Others,50.000000,50.000000


In [ ]:
pd.crosstab(df['Dietary Habits'], df['Depression'], normalize='index') * 100
# Probably useful, less healthy habits have an increasing prevalence of depression

Depression,0,1
Dietary Habits,,
Healthy,54.607241,45.392759
Moderate,43.977422,56.022578
Others,33.333333,66.666667
Unhealthy,29.272075,70.727925


In [ ]:
pd.crosstab(df['Have you ever had suicidal thoughts ?'], df['Depression'], normalize='index') * 100
# another strong correlation, having suicidal thoughts significantly correlates with having depression

Depression,0,1
Have you ever had suicidal thoughts ?,,
No,76.778917,23.221083
Yes,20.950385,79.049615


In [ ]:
pd.crosstab(df['Work/Study Hours'], df['Depression'], normalize='index') * 100
# not as strong a correlation as expected, but higher work/study hours does seem to (generally) increase chances of depression
# weird that people working 12 hours aren't more frequently depressed than people working 10?

Depression,0,1
Work/Study Hours,,
0.0,64.470588,35.529412
1.0,59.652174,40.347826
2.0,56.226415,43.773585
3.0,52.585034,47.414966
4.0,49.504337,50.495663
5.0,45.524691,54.475309
6.0,42.679128,57.320872
7.0,41.229385,58.770615
8.0,37.504982,62.495018


In [ ]:
pd.crosstab(df['Financial Stress'], df['Depression'], normalize='index') * 100
# another stronger correlation, having higher financial stress does increase chances of depression

Depression,0,1
Financial Stress,,
1.0,68.131224,31.868776
2.0,57.024303,42.975697
3.0,41.063911,58.936089
4.0,30.909091,69.090909
5.0,18.719285,81.280715


In [ ]:
pd.crosstab(df['Family History of Mental Illness'], df['Depression'], normalize='index') * 100
# not as much of a correlation as expected. Students w/ family history of mental illness do have higher rates, but not by much

Depression,0,1
Family History of Mental Illness,,
No,43.999167,56.000833
Yes,38.732134,61.267866


### Model Evaluation (Alok)

#### Dataset cleanup ####

Make a copy of the dataframe for cleanup (might want to do different things, so keep the original imported one)
Going through it one column at a time w/ notes

In [ ]:
df_1 = df.copy()


In [ ]:
# id: not sure why its numbered like that, probably not useful
#dropping it for now, might need it later
df_1 = df_1.drop('id', axis=1)

In [ ]:

## Gender: convert to one hot columns? maybe useful. Gender ratio is at 0.44, not so bad
# df["Gender"].value_counts()
df_1["Gender"] = df_1["Gender"].map({"Female": 1, "Male": 0})
df_1["Gender"] = df_1["Gender"].astype(bool)



In [ ]:
## Age: convert from float to int, no decimals. Mostly in 20s, maybe find an average
# df_1["Age"].value_counts()

df_1["Age"] = df_1["Age"].astype(int)


In [ ]:
# # City: convert to one hot columns. Put the couple dozen cities with one or two students in "other"
# # df_1["City"].value_counts()

# city_count = df_1["City"].value_counts()

# #getting cities that are in less than 10 data points
# threshold = 10
# rare_cities = city_count[city_count < threshold].index

# #replace the values in City where there's something in the rare cities list with other
# #got this partially from gpt, verify how it works
# df_1["City"] = df_1["City"].where(~df_1["City"].isin(rare_cities), "Other")
# city_count = df_1["City"].value_counts()
# # df_1["City"].value_counts()

# #get a data frame of one-hot columns of the cities
# cities_df = pd.get_dummies(df_1["City"], prefix="city")
# # cities_df_1.head()

# #then add it onto the side of the original dataframe
# df_1 = pd.concat([df_1, cities_df_1], axis=1)
# #remove the original cities column

# Actually, maybe too many cities to really help w/ one hot columns, disregarding it for now

df_1 = df_1.drop('City', axis=1)

In [ ]:
## Profession: not useful, barely anyone put anything other than "student"
# df_1["Profession"].value_counts()
#also dropping this one
df_1 = df_1.drop('Profession', axis=1)

In [ ]:
## Academic Pressure: convert to int. 3 is most popular, followed by 5, then 4.
# df_1["Academic Pressure"].value_counts()
df_1 = df_1.rename(columns={'Academic Pressure': 'Academic Pressure'})
df_1["Academic Pressure"] = df_1["Academic Pressure"].astype(int)

In [ ]:
# Work Pressure: Not useful. Only 3 people put anything other than 0
# df_1["Work Pressure"].value_counts()
df_1 = df_1.drop('Work Pressure', axis=1)

In [ ]:
## CGPA: Need to keep this as a float? There's 322 distinct values, but it seems useful.
## maybe separate it into brackets though
# df_1["CGPA"].value_counts()

#not touching this one for now, just putting it in lowercase
# df_1 = df_1.rename(columns={'CGPA': 'cgpa'})


In [ ]:
# Study Satisfaction: Convert to int. Suprisingly close values, save for people who put 0.0
# df_1["Study Satisfaction"].value_counts()

# dfdf_1 = df_1.rename(columns={'Study Satisfaction': 'Study Satisfaction'})
df_1["Study Satisfaction"] = df_1["Study Satisfaction"].astype(int)


In [ ]:
# Job Satisfaction: Not useful, assuming that 0.0 means N.A, and so many people put student
# df_1["Job Satisfaction"].value_counts()

df_1 = df_1.drop('Job Satisfaction', axis=1)

In [ ]:

## Sleep Duration: Convert to numeric values and one hot columns?
## it's in a weird format with things in buckets
# df["Sleep Duration"].value_counts()

#making one hot columns for these
sleep_df = pd.get_dummies(df_1['Sleep Duration'], prefix='sleep')
#adding them back onto the side, removing original
df_1 = pd.concat([df_1, sleep_df], axis=1)
#figure out some way to name them better later on

df_1 = df_1.drop('Sleep Duration', axis=1)

In [ ]:
# Dietary Habits: Need one hot columns. "Unhealthy" is most popular, but only by <500
# df_1["Dietary Habits"].value_counts()
diet_df = pd.get_dummies(df_1['Dietary Habits'], prefix='diet')
df_1 = pd.concat([df_1, diet_df], axis=1)
df_1 = df_1.drop('Dietary Habits', axis=1)


In [ ]:
## Degree: Lots of categories. Put into one hot columns? It'll make a lot of them
# df_1["Degree"].value_counts()
# changing this to gather up each term into larger categories than just one hot columns?
# degree_df = pd.get_dummies(df_1['Degree'], prefix='degree')
# df_1 = pd.concat([df_1, degree_df], axis=1)
df_1 = df_1.drop('Degree', axis=1)

# degree_categories = {
#     'Class 12': 'Others',
#     'B.Ed': 'Bachelors',
#     'B.Com': 'Bachelors',
#     'B.Arch': 'Bachelors',
#     'BCA': 'Bachelors',
#     'MSc': 'Masters',
#     'B.Tech': 'Bachelors',
#     'MCA': 'Masters',
#     'M.Tech': 'Masters',
#     'BHM': 'Bachelors',
#     'BSc': 'Bachelors',
#     'M.Ed': 'Masters',
#     'B.Pharm': 'Bachelors',
#     'M.Com': 'Masters',
#     'BBA': 'Bachelors',
#     'MBBS': 'Bachelors',
#     'LLB': 'Bachelors',
#     'BE': 'Bachelors',
#     'BA': 'Bachelors',
#     'M.Pharm': 'Masters',
#     'MD': 'Masters',
#     'MBA': 'Masters',
#     'MA': 'Masters',
#     'PhD': 'PhD',
#     'LLM': 'Masters',
#     'MHM': 'Masters',
#     'ME': 'Masters',
#     'Others': 'Others'

# }


In [ ]:
## Have you ever had suicidal thoughts ? : Convert to binary. Almost 60% say yes, which I guess could be worse
## Also rename to something more concise than all that

# df_1["Have you ever had suicidal thoughts ?"].value_counts()
# df_1 = df_1.rename(columns={'Have you ever had suicidal thoughts ?': 'suicidal_thoughts'})
df_1["Have you ever had suicidal thoughts ?"] = df_1["Have you ever had suicidal thoughts ?"].map({"Yes": True, "No": False})


In [ ]:
## Work/Study Hours: Most people are in the range of 10-12 hours. Convert to int.
# df_1["Work/Study Hours"].value_counts()
# df_1 = df_1.rename(columns={'Work/Study Hours': 'work_study_hours'})
df_1["Work/Study Hours"] = df_1["Work/Study Hours"].astype(int)

In [ ]:
# Financial Stress: Suprisngly close, with 5.0 on top. Convert to int.
# df = df.rename(columns={'Financial Stress': 'financial_stress'})
df_1["Financial Stress"].value_counts()
#wait, this column has two values that aren't filled in, it has 27898 entries?
#filling the handful of NA values with a 5, since it's already ahead by 900 or so
df_1["Financial Stress"] = df_1["Financial Stress"].fillna(5.0)

df_1["Financial Stress"] = df_1["Financial Stress"].astype(int)

In [ ]:
# Family History of Mental Illness: Also quite close numbers, convert to binary
#also rename this one
# df_1["Family History of Mental Illness"].value_counts()

# df_1 = df_1.rename(columns={"Family History of Mental Illness": "family_history"})
df_1["Family History of Mental Illness"] = df_1["Family History of Mental Illness"].map({"Yes": True, "No": False})

In [ ]:
# Depression: Main label. More people do than don't.
# df_1["Depression"].value_counts()

# df_1 = df_1.rename(columns={"Depression": "depression"})
df_1["Depression"] = df_1["Depression"].astype(bool)

#put depression at the end of the list so we can get at it easily
final_column = df_1.pop('Depression')
df_1['Depression'] = final_column


In [ ]:
#final name cleanup, put everything else in lowercase, as well as replacing any spaces
# df.columns = df.columns.str.lower()
# df.columns = df.columns.str.replace(" ", "_")
# df.columns = df.columns.str.replace("-", "_")

In [ ]:
df.info()
# df.head()


#### Evaluation Function ####
Split this up into pieces?

In [ ]:
def evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs):

    train_loss_tracker = []
    test_loss_tracker = []

    # adding this to also graph the accuracy in here
    train_accuracy_tracker = []
    test_accuracy_tracker = []

    for epoch in range(num_epochs):

        # set the model to train and put the accumulators to zero
        model.train()


        total_train_loss = 0.0
        train_correct = 0
        train_total = 0

        #do training loop
        for X_batch, y_batch in train_loader:

            X_batch = X_batch.to(device)
            y_batch.to(device)

            # do the forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_train_loss += loss.item()

            # do the  pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # get training accuracy as well
            # starting w/ predicted class


            train_predicted = (outputs > 0.5).float()
            train_correct += (train_predicted == y_batch).sum().item()
            train_total += y_batch.size(0)





        avg_train_loss = total_train_loss / len(train_loader)
        train_loss_tracker.append(avg_train_loss)

        train_accuracy = 100 * train_correct / train_total
        train_accuracy_tracker.append(train_accuracy)


        #do the test function afterwards

        model.eval()

        total_test_loss = 0.0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
           for X_batch, y_batch in test_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)


                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                total_test_loss += loss.item()


                # get accuracy for test as well
                test_predicted = (outputs > 0.5).float()
                test_correct += (test_predicted == y_batch).sum().item()
                test_total += y_batch.size(0)




        avg_test_loss = total_test_loss / len(test_loader)
        test_accuracy = 100 * test_correct / test_total
        test_loss_tracker.append(avg_test_loss)
        test_accuracy_tracker.append(test_accuracy)


        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% - "
              f"Test Loss: {avg_test_loss:.4f}, Test Acc: {test_accuracy:.2f}%")


    # Adding this from other doc to do both graphs at once



    plt.figure(figsize=(12, 5))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot( range(1, num_epochs + 1), train_loss_tracker, label='Training Loss', color='blue')
    plt.plot( range(1, num_epochs + 1), test_loss_tracker, label='Testing Loss', color='orange')
    plt.title('Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot( range(1, num_epochs + 1), train_accuracy_tracker, label='Training Accuracy', color='blue')
    plt.plot( range(1, num_epochs + 1), test_accuracy_tracker, label='Test Accuracy', color='orange')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()



    plt.tight_layout()
    plt.show()


#### Simple Neural Network 1 ####

Starting off w/ a simple neural network, looking at all the columns after cleanup

In [ ]:
X = df_1.iloc[:, 0:-1].values # get everything except for the last column, done the cleanup
y = df_1.iloc[:, -1].values   # only take the last one, since it has our label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#some weird conversion with going to numpy arrays
X_train = torch.tensor(X_train.astype(np.float32))
X_test = torch.tensor(X_test.astype(np.float32))


#similar kind of reshaping for binary classification
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 50

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

#### Networks

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        #doing it this way instead
        self.model = nn.Sequential(
            nn.Linear(18, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
      return self.model(x)

In [ ]:
model = SimpleNN()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

evaluate(model, train_loader, test_loader, loss_function, optimizer, num_epochs)

#### Simple Network 2 (Only opinion columns)

Testing a similar network with only the columns where the students gave opinion scores (1-5)

In [ ]:
df_1.info()

In [ ]:
df_2 = df_1[['Academic Pressure', 'Study Satisfaction', 'Work/Study Hours', 'Financial Stress', 'Depression']]

df_2.info()

In [ ]:
# just getting the basic numerical values for this one

X = df_2.iloc[:, 0:-1].values # get everything except for the last column, done the cleanup
y = df_2.iloc[:, -1].values   # only take the last one, since it has our label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46)

X_train = torch.tensor(X_train.astype(np.float32))
X_test = torch.tensor(X_test.astype(np.float32))


y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 50

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

X_train.size(1)

In [ ]:
class SimpleNN2(nn.Module):
    def __init__(self):
        super(SimpleNN2, self).__init__()
        #even simpler network
        self.model = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 6),
            nn.ReLU(),
            nn.Linear(6, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
      return self.model(x)

In [ ]:
model = SimpleNN2()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

evaluate(model, train_loader, test_loader, loss_function, optimizer, num_epochs)

Not sure what to make of higher accuracy for testing?
Regardless, similar results compared to shallower model

#### Simple Neural Network 3

Trying another model with just the correlated

### Model Testing (Wenxuan)

Read in the dataset without numerical values, checking if those are correlated enough.
Rather than editing it each time, saved as its own file

In [ ]:
df_alpha = pd.read_csv("StudentDepressionDataset_NumbersOnly.csv")
print("Dataset shape:", df_alpha.shape)



Check for NaN entries, and check how many rows have at least one

In [ ]:
# print(df.isnull().sum())  # Summarize how many NaNs in each column

# nan_rows = df[df.isnull().any(axis=1)]
# print("Rows with at least one NaN:")
# print(nan_rows)

Seperate out features & labels, everything except the last column

In [ ]:
X = df.iloc[:, :-1].values
# the last column is Depression_Status
y = df.iloc[:, -1].values

# Make sure y is in float format for BCEWithLogitsLoss
y = y.astype(np.float32)

# Train-test split
# picked a different random state to check it
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123, stratify=y
)

In [ ]:
# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Implementing a dataloader rather than _'s built in one

In [ ]:
# Create Dataset & DataLoader
class StudentDepressionDataset(Dataset):
    def __init__(self, features, labels):
        self.X = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.float32)  # note float for BCE

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# then run on our already split up data
train_dataset = StudentDepressionDataset(X_train, y_train)
test_dataset = StudentDepressionDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

NameError: name 'Dataset' is not defined

Model Definition, starting w/ our input features, and going down by 2 each layer, activating w/ ReLU each time

In [ ]:
# Define a model
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(16, 12)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(12, 8)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(8, 4)
        self.relu4 = nn.ReLU()
        self.out = nn.Linear(4, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.out(x)
        return x

Run the newly defined model

In [ ]:
input_dim = X_train.shape[1]
model = FeedForwardNN(input_dim)

# Use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Defining a test function w/ accumulators. Seeing how well it works over 200 epochs, which isn't small

In [ ]:

num_epochs = 200
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_features)           # shape: [batch_size, 1]
        loss = criterion(outputs.squeeze(), batch_labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_features.size(0)

    epoch_loss = running_loss / len(train_dataset)
    train_losses.append(epoch_loss)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            logits = model(batch_features).squeeze()    # shape: [batch_size]

            # Convert logits to predictions: pred = sigmoid(logits) > 0.5
            preds = torch.sigmoid(logits)  # in range [0, 1]
            predicted_labels = (preds >= 0.5).float()    # 0.0 or 1.0

            total += batch_labels.size(0)
            correct += (predicted_labels == batch_labels).sum().item()

    test_accuracy = 100.0 * correct / total
    test_accuracies.append(test_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {epoch_loss:.4f} | "
          f"Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
# Plot test accuracy
plt.plot(range(1, num_epochs+1), test_accuracies, label="Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Test Accuracy over Epochs")
plt.legend()
plt.show()

Plot Accuracy & Training loss to see the results

In [ ]:
# Plot training loss
plt.plot(range(1, num_epochs+1), train_losses, label="Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.legend()
plt.show()

### Model Testing (Gleb)

#### Dataset Examination/Cleanup

Take a look at the data (and the type of each column)

In [ ]:
shape = df.shape
dtypes = df.dtypes
sample_rows = df.head()

shape, dtypes, sample_rows

Doing some data cleaning first

In [ ]:
# drop the id column
df_cleaned = df.drop(columns=["id"])

#fill the missing financial stress values
df_cleaned["Financial Stress"].fillna(df_cleaned["Financial Stress"].median(), inplace=True)

Make a map to convert sleep duration into something the model can handle

In [ ]:
sleep_map = {
    "Less than 5 hours": 4.5,
    "5-6 hours": 5.5,
    "7-8 hours": 7.5,
    "More than 8 hours": 9.0
}
df_cleaned["Sleep Duration"] = df_cleaned["Sleep Duration"].map(sleep_map)

Convert other non numerical values.
This loop goes through each column in the list of categorical columns. <br>
It uses `pd.factorize()` to convert string categories into integer codes
e.g., "Male", "Female" → 0, 1. <br>

In [ ]:
factorize_mappings = {}

categorical_columns = [
    "Gender", "City", "Profession", "Dietary Habits", "Degree",
    "Have you ever had suicidal thoughts ?", "Family History of Mental Illness"
]

for col in categorical_columns:
    df_cleaned[col], uniques = pd.factorize(df_cleaned[col])  # convert to number
    factorize_mappings[col] = dict(enumerate(uniques)) # save to factorize map

factorize_mappings # display mappings

As well, remove any null values, and scale the features, before finally splitting the dataset

In [ ]:
df_cleaned = df_cleaned.dropna()

In [ ]:
X = df_cleaned.drop(columns=["Depression"])
y = df_cleaned["Depression"]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

#### Model Preparation

Using tensorflow, check with the logistical regression model first

In [ ]:
# initialize, then train model
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# get predictions
y_pred = logreg.predict(X_test)

# get accuracy score by comparing predictions w/ labels
print("Accuracy:", accuracy_score(y_test, y_pred))

Trying gradient boost model next

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_gb))

Also checking a simple DL model

In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Evaluate model
y_pred_xgb = xgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_xgb))

Finally, setting up a simple DL model

In [ ]:
model0 = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

# Compile the model
model0.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model0.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),  # equivalent to your test loop
    epochs=50,
    batch_size=32,
    verbose=1
)

#### Model Testing

Plot accuracy/loss for training and test sets

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 1 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model 1 Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred_probs = model0.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")
print(accuracy_score(y_test, y_pred))

Trying again w/ a deeper model

In [ ]:
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=32,
                    verbose=1)

Check it's accuracy as well:

In [ ]:
y_pred_probs = model.predict(X_test)    # test
y_pred = (y_pred_probs > 0.5).astype("int32")
print(accuracy_score(y_test, y_pred))

Then plot training/validation accuracy and loss values

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 2 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model 2 Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()



*   The model is learning and fitting the data
*   There isn't any real gain on the validation set
* Training Loss is decreasing, validation loss is bouncing around, but not improving
* Possibly some overfitting as well






Trying an even more complicated model next, with early stopping

In [ ]:
model2 = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Define early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model2.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model2.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# getting the new model's accuracy as well
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")
print(accuracy_score(y_test, y_pred))

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 3 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model 3 Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

Train accuracy starts 78% and climbs to 85.5% → nice upward trend. <br>
Validation ccuracy starts high (84.2%) and remains stable <br>

Train loss clearly decreases → model is optimizing well. <br>
Validation flat. <br>




#### Testing with Keras Tuner

Trying again with Keras Tuner, a library that automatically searches for a better architecture (number of layers, neurons per layer, dropout, learning rate, etc)

Defining a model-building function that Keras will call w/ different hyperparameters while testing different models

In [ ]:
def build_model(hp):
    model = models.Sequential() # seqeuntial model FNN
    model.add(layers.Input(shape=(X_train.shape[1],)))  # input layer

    # Tune number of layers and units
    for i in range(hp.Int('num_layers', 1, 3)):    #Number of layers: 1, 2, or 3 (randomly chosen)
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32), #Units per layer: 32, 64, 96, or 128
            activation='relu'
        ))
        model.add(layers.Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1))) #Dropout rate: Between 0.2 and 0.5 in 0.1 steps

    model.add(layers.Dense(1, activation='sigmoid')) # activation sigmoid

    # Tune learning rate
    lr = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001]) # learning rate choices

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),    #compile the model
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

Defining the tuner, which runs through different cominations (10 of them)

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Try 10 different combinations
    executions_per_trial=1,
    directory='kt_dir',
    project_name='depression_prediction'
)

Call the tuner to search throughout the models

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

tuner.search(
    X_train, y_train,
    epochs=30,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

After getting the best model from the search, print information on it

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(1)[0]

print("Best hyperparameters:")
print(best_hps.values)

# Evaluate on test set
loss, acc = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

As well, display the results from the other trials the tuner ran:

In [ ]:
tuner_results = tuner.oracle.get_best_trials(num_trials=10)

# Extract validation accuracies
accuracies = [trial.metrics.get_last_value('val_accuracy') for trial in tuner_results]
names = [f'Trial {i+1}' for i in range(len(tuner_results))]

# Plot
plt.figure(figsize=(10, 5))
plt.bar(names, accuracies, color='skyblue')
plt.ylim(0.83, 0.86)  # Zoom in for more detail
plt.ylabel('Validation Accuracy')
plt.xlabel('Trial')
plt.title('Validation Accuracy Across Hyperparameter Trials')
plt.xticks(rotation=45)

# Add value labels above bars
for i, acc in enumerate(accuracies):
    plt.text(i, acc + 0.0005, f"{acc:.4f}", ha='center', va='bottom')

plt.grid(True, axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

NameError: name 'tuner' is not defined

#### ChatGPT Prompting (Gleb)

As well, giving an LLM the same information for a given student, and seeing what it's response is

First, testing with just one prompt

In [ ]:
row = df.iloc[0] # get the first row

In [ ]:
openai.api_key = "" # removed this for now, need to save it as a secret

#### Test Prompt

Creating a prompt asking about that one student

In [ ]:
content = f"""
You are a mental health prediction model. Given the following data, your job is to classify them as either 1 for depressed or 0 for not depressed.

Here is the student information:
- Gender: {row['Gender']}
- Age: {row['Age']}
- City: {row['City']}
- Profession: {row['Profession']}
- Academic Pressure: {row['Academic Pressure']}
- Work Pressure: {row['Work Pressure']}
- CGPA: {row['CGPA']}
- Study Satisfaction: {row['Study Satisfaction']}
- Job Satisfaction: {row['Job Satisfaction']}
- Sleep Duration: {row['Sleep Duration']}
- Dietary Habits: {row['Dietary Habits']}
- Degree: {row['Degree']}
- Have they had suicidal thoughts before? {row['Have you ever had suicidal thoughts ?']}
- Work/Study Hours: {row['Work/Study Hours']}
- Financial Stress: {row['Financial Stress']}
- Family history of mental illness: {row['Family History of Mental Illness']}

Respond with only one word: either 1 or 0.
"""

In [ ]:
# get a response to our query from OpenAI
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": content}],
    temperature=0.0
)  # get open AI response

Print the response we get:

In [ ]:
print("Prediction:", response.choices[0].message.content.strip()) # Print the response

#### Larger Scale Prompting

After that, let's try using ChatGPT to predict the first hundred rows in our dataset

In [ ]:
# Store predictions
gpt_preds = []

# Loop through the first 100 rows
for idx, row in df.head(100).iterrows():
    content = f"""
              You are a mental health prediction model. Given the following student's data, your job is to classify them as either 1 for depressed or 0 for not depressed.

              Here is the student information:
              - Gender: {row['Gender']}
              - Age: {row['Age']}
              - City: {row['City']}
              - Profession: {row['Profession']}
              - Academic Pressure: {row['Academic Pressure']}
              - Work Pressure: {row['Work Pressure']}
              - CGPA: {row['CGPA']}
              - Study Satisfaction: {row['Study Satisfaction']}
              - Job Satisfaction: {row['Job Satisfaction']}
              - Sleep Duration: {row['Sleep Duration']}
              - Dietary Habits: {row['Dietary Habits']}
              - Degree: {row['Degree']}
              - Have they had suicidal thoughts before? {row['Have you ever had suicidal thoughts ?']}
              - Work/Study Hours: {row['Work/Study Hours']}
              - Financial Stress: {row['Financial Stress']}
              - Family history of mental illness: {row['Family History of Mental Illness']}

              Respond with only one word: either 1 or 0.
              """
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": content}],
            temperature=0.0
        )
        prediction = response.choices[0].message.content.strip()
    except Exception as e:
        prediction = "error"
        print(f"Row {idx} failed: {e}")

    print(f"Row {idx} prediction: {prediction}")
    gpt_preds.append(prediction)

    time.sleep(1)  # avoid rate limits

#### Evaluation

Finally, let's compare our results with the actual predictions, and store the results in a dataframe

In [ ]:
# Compare with actual labels
actual = df.loc[:99, "Depression"].astype(str)

# Create a comparison DataFrame
results_df = pd.DataFrame({
    "Actual": actual,
    "GPT_Prediction": gpt_preds
})

# Filter out errors before computing accuracy
clean_df = results_df[results_df["GPT_Prediction"].isin(["0", "1"])]
len(clean_df["GPT_Prediction"])
accuracy = (clean_df["Actual"] == clean_df["GPT_Prediction"]).mean()

In [ ]:
print("Accuracy:", accuracy)
results_df.head(10)

### Suggestions

In [ ]:
# Suggestions
#precision recall, f1 score to improve loss
# Going to try and get llama or deepseek running in here to answer prompts

#### Llama Prompting V4

Trying this again:
https://colab.research.google.com/github/5aharsh/collama/blob/main/Ollama_Setup.ipynb




In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,688 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)


In [ ]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   1% ▕▏  11 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   1% ▕▏  21 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   2% ▕▏  46 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   3% ▕▏  69 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   4% ▕▏  81 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   5% ▕▏ 104 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   6% ▕▏ 119 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff

In [ ]:
!pip install langchain-ollama

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from IPython.display import Markdown

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")
# change this to deepseek easily enough

chain = prompt | model


practicing reading from a file first

In [ ]:
import pandas as pd
import os
from time import sleep
from google.colab import files

In [ ]:
# read in a saved model predictions file first
uploaded = files.upload()

In [ ]:
df = pd.read_csv("Student Depression Dataset.csv")


In [ ]:
question_list = []

# not taking all the information
# saw above that city, and job related columns aren't useful, not taking them

# just trying to ask the first few them
N = 150

for i in range(0,N):
    question = (f"""
You are a mental health prediction model. Given the following student's data, your job is to classify them as either 1 for depressed or 0 for not depressed.
- Gender: {df['Gender'].iloc[i]}
- Age: {df['Age'].iloc[i]}
- Academic Pressure: {df['Academic Pressure'].iloc[i]}
- CGPA: {df['CGPA'].iloc[i]}
- Study Satisfaction: {df['Study Satisfaction'].iloc[i]}
- Sleep Duration: {df['Sleep Duration'].iloc[i]}
- Dietary Habits: {df['Dietary Habits'].iloc[i]}
- Degree: {df['Degree'].iloc[i]}
- Have they had suicidal thoughts before? {df['Have you ever had suicidal thoughts ?'].iloc[i]}
- Work/Study Hours: {df['Work/Study Hours'].iloc[i]}
- Financial Stress: {df['Financial Stress'].iloc[i]}
- Family history of mental illness: {df['Family History of Mental Illness'].iloc[i]}
Please respond with only one word. Either 1 (for depressed), or 0 (for not depressed)
""")

    question_list.append(question)

print("Generated a list of questions for the model, the first", len(question_list), "of them")


Generated a list of questions for the model, the first 150 of them


In [ ]:
output_file = "llama_predictions.csv"

In [ ]:
# check if file has already been created/uploaded
if os.path.exists(output_file):
  print("predictions file does exist, loaded llama_predictions.csv")
  # if it does, copy it into a dataframe
  llama_results_df = pd.read_csv(output_file)
  # and use the length of the file to figure out what index to start from

else:
  # if not, it either hasn't been uploaded, or we're starting fresh
  results_df = pd.DataFrame(columns=["Index" ,"Response"])
  # so put the index at 0
  print("no file found, starting from index 0")


no file found, starting from index 0


Like above, make a list of questions for asking the model

In [ ]:
# get a list of indexes it already asked about
existing_indices = set(results_df["Index"].values)

print(existing_indices)

[]


In [ ]:
# run the model
for i in range(len(question_list)):
  # check if the current index has already been asked
  if i in results_df["Index"].values:
    print(f"Skipping index {i}, already asked")
    continue

  print(f"Asking question for {i}")
  question = question_list[i]

  try:
      response = chain.invoke({"question": question})
  except Exception as e:
      response = "ERROR"


  # wish I could mark these as bools like another language
  contains_1 = "1" in response
  contains_0 = "0" in response

  # if the response doesn't have a 1 or (exclusively)
  if contains_1 ^ contains_0:
    status = "VALID"

  else:
    status = "INVALID"
    response = "ERROR"


  # add result to dataframe, then save it
  results_df.loc[len(results_df)] = [i, response]
  print(f"Saved response at index {i} to file, {status}")
  results_df.to_csv(output_file, index=False)
  # not sure if waiting a bit helps, apparently it might prevent overloading?
  # sleep(0.5)





Asking question for 0
Saved response at index 0 to file, VALID
Asking question for 1
Saved response at index 1 to file, VALID
Asking question for 2
Saved response at index 2 to file, VALID
Asking question for 3
Saved response at index 3 to file, INVALID
Asking question for 4
Saved response at index 4 to file, VALID
Asking question for 5
Saved response at index 5 to file, VALID
Asking question for 6
Saved response at index 6 to file, VALID
Asking question for 7
Saved response at index 7 to file, VALID
Asking question for 8
Saved response at index 8 to file, VALID
Asking question for 9
Saved response at index 9 to file, VALID
Asking question for 10
Saved response at index 10 to file, VALID
Asking question for 11
Saved response at index 11 to file, VALID
Asking question for 12
Saved response at index 12 to file, VALID
Asking question for 13
Saved response at index 13 to file, VALID
Asking question for 14
Saved response at index 14 to file, VALID
Asking question for 15
Saved response at in

In [ ]:



  # # check if response contains a 1 or 0, if it doesn't
  # if response.find("1") != -1 or response.find("0") != -1:
  #   # if it is alright, add the response to the dataframe and save it to the file
  #   results_df.loc[len(results_df)] = [i, response]
  #   results_df.to_csv(output_file, index=False)
  #   print("saved response to index:", i)
  # else:
  #   # then the llm didn't respond right, keep going, get another time
  #   print("error on index:", i, "continuing on")
  #   # just put an error in there for now
  #   results_df.loc[len(results_df)] = [i, "ERROR"]
